# Phylogenetic Inference with across site rate variation

This notebook gives an example on how to do phylogenetic inference using the PNUTS algorithm and across site rate variation.

First the MCPhylo package and the Random package are loaded. And the data is loaded.

In [ ]:
using MCPhylo;
using Random;
Random.seed!(1234);

tree, data = make_tree_with_data("Example.nex");

data_dictionary = Dict{Symbol, Any}(
  :data => data
);

Create a model which uses discrete gamma rates for across site rate variaton.

In [ ]:
model =  Model(
    data = Stochastic(3, (tree, eq_freq, rates) ->  PhyloDist(tree, eq_freq, [1.0], rates, Restriction), false, false),
    eq_freq = Stochastic(1, () -> Dirichlet(2,1)),
    tree = Stochastic(Node(), () -> CompoundDirichlet(1.0,1.0,0.100,1.0), true),
    rates = Logical(1, (a)->discrete_gamma_rates(a,a,4)),
    a = Stochastic(()->Exponential(), true)
     )

Set the samplers for the tree, the equilibrium frequencies and the parameter for the discrete gamma rates.

In [ ]:
scheme = [PNUTS(:tree, target=0.7, targetNNI=4),
          SliceSimplex(:eq_freq),
          Slice(:a, 1.0)
          ]

setsamplers!(model, scheme);

inits = [ Dict{Symbol, Union{Any, Real}}(
    :tree => tree,
    :eq_freq=> rand(Dirichlet(2,1)),
    :data => data_dictionary[:data],
    :a => rand()
    ),
    ];


Run the model.

In [ ]:
sim = mcmc(model, data_dictionary, inits, 5000, burnin=2500,thin=5, chains=1, trees=true)